<a href="https://colab.research.google.com/github/nunzio998/Chat_P2P/blob/main/physical_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physical Anomaly Detection

## Import Librerie

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape

## Funzione per caricamento e segmentazione dataset
Il dataset viene suddiviso in intervalli di un minuto in modo da addestrare la rete su intervalli di campioni senza anomalie, in modo da poter riconoscere in fase di test quando un intervallo contiene invece un'anomalia.

In [2]:
# 1.
def dataset_load_and_segmentation(path):
    # Carico il CSV con diversa codifica
    df_norm = pd.read_csv(path, sep='\t', encoding='utf-16')

    df_norm = df_norm.drop(columns=["Label"])

    df_norm['Time'] = pd.to_datetime(df_norm['Time'])

    # Ordino per timestamp nel caso non siano già ordinati
    df_norm = df_norm.sort_values(by='Time')

    # Definisco la durata della finestra in due minuti
    window_duration = pd.Timedelta(minutes=1)

    # Lista per i segmenti
    segments = []
    start_time = df_norm['Time'].iloc[0]

    while start_time < df_norm['Time'].iloc[-1]:
        end_time = start_time + window_duration
        segment = df_norm[(df_norm['Time'] >= start_time) & (df_norm['Time'] < end_time)]
        if len(segment) > 0:
            segments.append(segment.drop(columns=['Time']).values)
        start_time = end_time

    # print(len(segments))
    # for seg in segments:
    #     print(len(seg))

    # Mantengo solo i segmenti che hanno lunghezza pari a 60 o 120 in base a se divido in intervalli di 2 o 1 minuti.
    uniform_segments = [segment for segment in segments if len(segment) == 60]

    # print("\n------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")
    #
    # print(len(uniform_segments))
    # for seg in uniform_segments:
    #     print(len(seg))

    return uniform_segments

## Funzione per il Preprocessing

In [3]:
# 2.
def preprocessing(segments):
    """
    1) Accorpo tutti i segmenti in un unico numpy array per poter applicare la normalizzazione.
    2) Normalizzo.
    3) Divido di nuovo i segmenti normalizzati nel numero originario dei segmenti in input.
    4) Ritorno i segmenti normalizzati e lo scaler utilizzato.
    :param segments:
    :return:
    """
    # 1)
    segments_array = np.vstack(segments)

    # print(f"Array Unico: {segments_array}")
    # print(f"Tipologia: {type(segments_array)}")
    # print(f"Lunghezza: {len(segments_array)}")
    # print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 2)
    scaler = StandardScaler()
    segments_scaled = scaler.fit_transform(segments_array)

    # print(f"Segmenti Normalizzati: {segments_scaled}")
    # print(f"Tipologia: {type(segments_scaled)}")
    # print(f"Lunghezza: {len(segments_scaled)}")
    # print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 3)
    segments_scaled_split = np.array_split(segments_scaled, len(segments))

    # print(f"Segmenti Normalizzati e Ricostruiti: {segments_scaled_split}")
    # print(f'Tipologia: {type(segments_scaled_split)}')
    # print(f"Lunghezza: {len(segments_scaled_split)}")
    # print("\n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")

    # 4)
    return segments_scaled_split, scaler

## Funzione per il Training

In [4]:
# 3.
def building_and_training(segments_scaled_split):
    # Definisco l'input shape
    input_shape = segments_scaled_split[0].shape  # Forma di un segmento

    # Definisco la struttura dell'autoencoder
    input_layer = Input(shape=input_shape)
    x = Flatten()(input_layer)
    x = Dense(64, activation="relu")(x)
    encoded = Dense(32, activation="relu")(x)
    x = Dense(64, activation="relu")(encoded)
    x = Dense(np.prod(input_shape), activation="sigmoid")(x)
    decoded = Reshape(input_shape)(x)

    # Creo il Modello
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer="adam", loss="mse")

    # Addestramento dell'autoencoder sui dati "normali", senza anomalie
    autoencoder.fit(np.array(segments_scaled_split), np.array(segments_scaled_split), epochs=500, batch_size=8,
                    shuffle=True, validation_split=0.1)

    return autoencoder


## Calcolo dell'Errore di Ricostruzione

In [5]:
# 4.
def get_rebuilding_error(autoencoder, segments_scaled_split):
    # Ricostruzione dei dati di addestramento
    reconstructed_train = autoencoder.predict(np.array(segments_scaled_split))
    mse_train = np.mean(np.power(segments_scaled_split - reconstructed_train, 2), axis=(1, 2))

    # Imposto il threshold al 95° percentile dell'errore
    threshold = np.percentile(mse_train, 95)
    print("Soglia di errore di ricostruzione:", threshold)

    return threshold

## Test del modello

In [22]:
# 5.
def testing(path, scaler, autoencoder, threshold):
    # Carico il CSV con diversa codifica
    df_anomalous = pd.read_csv(path, sep='\t', encoding='utf-16')
    df_anomalous = df_anomalous.drop(columns=["Label"])

    df_anomalous['Time'] = pd.to_datetime(df_anomalous['Time'])
    df_anomalous = df_anomalous.sort_values(by='Time')

    # Segmentazione del file di test in blocchi di tot minuti
    test_segments = []
    start_time = df_anomalous['Time'].iloc[0]

    # Definisco la finestra di tot minuti
    window_duration = pd.Timedelta(minutes=1)

    while start_time < df_anomalous['Time'].iloc[-1]:
        end_time = start_time + window_duration
        segment = df_anomalous[(df_anomalous['Time'] >= start_time) & (df_anomalous['Time'] < end_time)]
        if len(segment) > 0:
            test_segments.append(segment.drop(columns=['Time']).values)
        start_time = end_time

    # Normalizzazione dei segmenti di test usando lo scaler già addestrato
    test_segments_scaled = [scaler.transform(segment) for segment in test_segments]

    # Mantengo solo i segmenti che hanno lunghezza pari a 60 o 120 in base a se divido in intervalli di tot minuti.
    uniform_segments = [segment for segment in test_segments_scaled if len(segment) == 60]

    uniform_segments = uniform_segments[0:5]
    print(len(uniform_segments))

    for seg in uniform_segments:
        print(seg)

    # Ricostruzione e calcolo dell'errore per ogni segmento di test
    reconstructed_test = autoencoder.predict(np.array(uniform_segments))
    mse_test = np.mean(np.power(uniform_segments - reconstructed_test, 2), axis=(1, 2))

    # Identificazione delle anomalie nei blocchi di test
    anomalies = mse_test > threshold
    print("Numero di blocchi anomali rilevati:", np.sum(anomalies))
    print(f"Blocchi anomali: {anomalies}")

## Esecuzione del processo

In [7]:
segments = dataset_load_and_segmentation("phy_norm.csv")

In [8]:
segments_scaled_split, scaler = preprocessing(segments)

In [ ]:
autoencoder = building_and_training(segments_scaled_split)

print(autoencoder)

In [10]:
threshols = get_rebuilding_error(autoencoder, segments_scaled_split)
print(threshols)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Soglia di errore di ricostruzione: 0.4955329067150167
0.4955329067150167


In [ ]:
testing("phy_att_3.csv", scaler, autoencoder, threshols)